# Example: Sudoku augmented design

We start by importing required modules

In [ ]:
using SudokuPlantDesign
using DataFrames
using XLSX
using PyPlot

## 1) Generate (optimized) Sudoku configuration

As explained in `sudoku_basic`, we generate a new configuration `conf` which is divided into 2 horizontal and 4 vertical blocks of dimensions `9` x `2` respectively. In total, there are `3` different checks in the configuration. All fields which are neither checks nor missing (empty) plots are defined as entries - these are the unreplicated entries which are augmented by replicated checks. We leave the plot `[1,3]` and `[1,4]` empty. Then, checks are initialized randomly so that there are `119` entries in total.

The configuration is plotted to show the initial starting point before optimization. 

In [ ]:
conf = get_configuration([9,9],[2,2,2,2],3)

empty_plots!(conf, 1:1,3:4)
initialize_entries!(conf, 119)

show_configuration(conf, zoom=0.2)

mkpath("output/")
savefig("output/augmented_design_checks_initial.pdf")

We use a standard selection of cost functions, as used in the `sudoku_basic` example.

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_num_checks_equal_per_type(conf) +
            K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

In the Sudoku-augmented design, we use 3 updates -- changing check labels, swapping checks with each other and also swapping checks with entries.

In [ ]:
updates = [UpdateNewCheckLabel(),UpdateSwapCheckCheck(),UpdateSwapCheckEntry()]

The optimization is run in the following

In [ ]:
costs = optimize_design!(
    conf,
    updates,
    K_indiv,
    500000
);

and the resulting configuration is visually checked by plotting

In [ ]:
print_info(conf)


show_configuration(conf)
mkpath("output/")
savefig("output/augmented_design_checks_final.pdf")

## 2) Save design data with field plan

With an optimized configuration `conf` at hand, one can proceed to create a field plan. For this, dataframes with the data for checks and entries are required.

In the case of our example, an Excel sheet with tabular input data `input_augmented.xlsx` containing sheets `checks` and `entries` is read in and converted into a dataframe. Each sheet contains the name of the entry in the first column, and in following columns further properties of the entries can be added to be transferred to the final output file and field plan. 

In [ ]:
entrydata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "entries")));
replace!.(eachcol(entrydata), "missing" => "NA");

In [ ]:
checkdata = string.(DataFrame(XLSX.readtable("input_augmented.xlsx", "checks")));
replace!.(eachcol(checkdata), "missing" => "NA");

To create a field plan, a labeled check configuration is created (based on the optimized configuration `conf` from the previous step). In this labeled configuration, both indices (the linear positions of the plants) as well as labels are set. The configuration is finally shown.

In [ ]:
lconf = LabeledCheckConfiguration(conf)

fill_indices_snake_y!(lconf, 1,1, index_for_empty=false)
fill_labels!(lconf, checkdata, entrydata)

show_configuration(lconf, check_labels=true)
mkpath("output/")
savefig("output/augmented_design_final_design.pdf")

For exporting, the data of this optimized Sudoku-augmented design can now be converted back into a dataframe

In [ ]:
df = get_dataframe(lconf)

The dataframe can be modified in julia before exporting, here all generic property columns are renamed to the the column names of the checkdata file.

In [ ]:
for (i,name) in enumerate(names(checkdata)[2:end])
    rename!(df,Symbol("property_"*string(i)) => Symbol(name))
end

df

Additional columns which hold the same data for all rows are inserted

In [ ]:
df[:, :year]       .= 2023
df[:, :extra_info] .= "myextrainfo"

df

finally, the dataframe is written back into an Excel file.

In [ ]:
mkpath("output/")
XLSX.writetable("output/augmented_design_final_design.xlsx", collect(eachcol(df)), names(df),overwrite=true)